# Banknote Authentication

## Data Set Information:

Les données ont été extraites des images qui ont été prises pour l'évaluation d'une procédure d'authentification pour les billets de banque.<br>(Owner of database: Volker Lohweg, University of Applied Sciences, Ostwestfalen-Lippe)
<ul>
<li>Nombre d'instances : 1372</li>
<li>Type : réel</li>
<li>Nombre de caractéristiques : 4</li>

<ul>
<li>X1 : variance </li>
<li>X2 : skewness </li>
<li>X3 : curtosis </li>
<li>X4 : entropy </li>
</ul>

<li>Tâche : Classification</li>
</ul>

## Architecture

<img src="images/nn_2layers.png">

<img src=images/nn_2layers_1.png>

# HEADERS

In [1]:
import tensorflow as tf
import pandas as pan

In [2]:
learning_rate = 0.01
training_epochs = 1000
step_display = 100

# 1. DATA

### 1.1. Load Data

In [3]:
data = pan.read_csv('data/money.csv')

In [4]:
data[0 :5]

,X1,X2,X3,X4,Label1,Label2
0,-0.025314,-0.17383,-0.11339,1.21980,1,0
1,5.807000,5.00970,-2.23840,0.43878,0,1
2,-2.434900,-9.24970,8.99220,-0.50001,1,0
3,-1.693600,2.78520,-2.18350,-1.92760,1,0
4,0.636550,5.20220,-5.21590,-6.12110,1,0


### 1.2. Train data

In [5]:
x_train = data.loc[0:1000, ['X1', 'X2', 'X3', 'X4']].as_matrix()
y_train = data.loc[0:1000, ['Label1', 'Label2']].as_matrix()

### 1.3. Test data

In [6]:
x_test = data.loc[1001: len(data), ['X1', 'X2', 'X3', 'X4']].as_matrix()
y_test = data.loc[1001: len(data), ['Label1', 'Label2']].as_matrix()

# 2. BUILD GRAPH

### 2.1. Placholders

In [7]:
x = tf.placeholder(tf.float32, shape=[None, 4], name='X')
y = tf.placeholder(tf.float32, shape=[None, 2], name='Y')

### 2.2. Model

2.2.1. Layers

In [8]:
def layer(x, size_in, size_out, name="Layer"):    
    with tf.name_scope(name):        
        w = tf.Variable(tf.zeros([size_in, size_out]), name='weight')
        b = tf.Variable(tf.zeros([size_out]), name='biais')
        z = tf.matmul(x, w) + b
        a = tf.sigmoid(z)
        tf.summary.histogram("weight", w)
        tf.summary.histogram("biais", b)
        tf.summary.histogram("Activation", a)
        return a

In [9]:
Layer_1 = layer(x      , 4, 4)
Layer_2 = layer(Layer_1, 4, 4)
Layer_3 = layer(Layer_2, 4, 2)

In [10]:
with tf.name_scope("softmax"):
    y_pred = tf.nn.softmax(Layer_3)

2.2.2. Cost function

In [11]:
with tf.name_scope("Error"):
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y * tf.log(y_pred)))
    tf.summary.scalar("CrossEnt", cross_entropy)

2.2.3. Optimizer

In [12]:
with tf.name_scope("Train"):
    optimiser = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

2.2.4. Accuracy

In [13]:
 with tf.name_scope('Accuracy'):    
    correct_prediction = tf.equal(tf.argmax(y_pred,1), tf.argmax(y,1))
    final_acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))*100
    tf.summary.scalar("Accuracy", final_acc)

# 3.SESSION

### 3.1. Start session

In [14]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [15]:
merged_summary = tf.summary.merge_all()
writer = tf.summary.FileWriter("./logs/Demo_3")
writer.add_graph(sess.graph)

### 3.2. Training

In [16]:
for step in range(training_epochs+1):
    _, cost = sess.run([optimiser, cross_entropy], feed_dict={x : x_train, y: y_train})
    
    s =  sess.run(merged_summary, feed_dict={x : x_train, y: y_train})
    writer.add_summary(s,step) 
  
    if step % step_display == 0 :
        print(step,'::', cost)

0 :: 693.839
100 :: 547.221
200 :: 386.258
300 :: 350.428
400 :: 338.569
500 :: 332.886
600 :: 329.638
700 :: 327.575
800 :: 326.17
900 :: 325.163
1000 :: 324.413


### 3.4. Compute accuracy

In [17]:
print ('Accuracy = {:05.2f}'.format(sess.run(final_acc,feed_dict={x: x_test, y:y_test})),'%')

Accuracy = 99.19 %
